<a href="https://colab.research.google.com/github/hansjang/Py_Radar/blob/main/%EB%8D%B0%EC%8B%B8%EB%9D%BC%EB%A9%B4Part6_P218.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/hansjang/Py_Radar.git

import shutil
shutil.move('Py_Radar/data', '/content/data')

# 폴더와 그 안의 모든 파일을 삭제
# !rm -rf /content/Py_Radar
# !rm -rf /content/data

Cloning into 'Py_Radar'...
remote: Enumerating objects: 192, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 192 (delta 9), reused 0 (delta 0), pack-reused 166 (from 1)
Receiving objects: 100% (192/192), 6.87 MiB | 14.03 MiB/s, done.
Resolving deltas: 100% (95/95), done.


'/content/data'

# K-최근접 이웃(KNN)

## KNN 분류

In [4]:
# KNeighborsClassifier
import numpy as np
import pandas as pd

liver = pd.read_csv('/content/data/indian_liver_patient.csv')
display(liver.head())
display(liver['Dataset'].value_counts())

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset
0,65,Female,0.7,0.1,187,16,18,6.8,3.3,0.90,1
1,62,Male,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,Male,7.3,4.1,490,60,68,7.0,3.3,0.89,1
3,58,Male,1.0,0.4,182,14,20,6.8,3.4,1.00,1
4,72,Male,3.9,2.0,195,27,59,7.3,2.4,0.40,1


,count
Dataset,
1,416
2,167


In [7]:
# 전처리
# 범주형 변수 Gender를 One-hot Encoding
liver.Gender = np.where(liver.Gender == 'Male', 1, 0)

# 결측치
# print(liver.isnull().sum())
liver.dropna(axis=0, inplace=True)
print(liver.isnull().sum())

Age                           0
Gender                        0
Total_Bilirubin               0
Direct_Bilirubin              0
Alkaline_Phosphotase          0
Alamine_Aminotransferase      0
Aspartate_Aminotransferase    0
Total_Protiens                0
Albumin                       0
Albumin_and_Globulin_Ratio    0
Dataset                       0
dtype: int64


In [10]:
# 데이터 분할
from sklearn.model_selection import train_test_split
x = liver[liver.columns.difference(['Dataset'])]
y = liver['Dataset']

train_x, test_x, train_y, test_y = train_test_split(x, y, stratify=y,
                                                    train_size = 0.7,
                                                    random_state = 1)
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(405, 10) (405,) (174, 10) (174,)


In [12]:
# 모델 적용
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors = 15, weights = 'uniform')
# n_neighbors = 15 : 새로운 데이터 포인트를 분류할 때 그 점에서 가장 가까운 15개의 이웃을 참조해요
# weights = 'uniform' : 모든 이웃이 동일한 가중치를 가져요.
#                       즉, 거리에 상관없이 15개의 이웃들이 모두 동일한 중요성을 가집니다.
# weights = 'distance' : 가까운 이웃에게 더 큰 가중치를 부여하고, 멀리 있는 이웃은 가중치를 줄입니다.
#                        즉, 가까운 이웃이 더 많은 영향을 미치도록 합니다.
clf.fit(train_x, train_y)

KNeighborsClassifier(n_neighbors=15)

In [14]:
# 성능 평가
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score

pred = clf.predict(test_x)

test_cm = confusion_matrix(test_y, pred)
test_acc = accuracy_score(test_y, pred)
test_prc = precision_score(test_y, pred)
test_rcll = recall_score(test_y, pred)
test_f1 = f1_score(test_y, pred)

print(test_cm)
print('\n')
print('정확도\t{}%'.format(round(test_acc * 100, 2)))   # 정확도	67.82%
print('정밀도\t{}%'.format(round(test_prc * 100, 2)))   # 정밀도	73.61%
print('재현율\t{}%'.format(round(test_rcll * 100, 2)))  # 재현율	85.48%
print('F1점수\t{}%'.format(round(test_f1 * 100, 2)))    # F1점수	79.1%

[[106  18]
 [ 38  12]]


정확도	67.82%
정밀도	73.61%
재현율	85.48%
F1점수	79.1%


In [15]:
# 만약 가중치를 distance로 했을때?
# 모델 적용
from sklearn.neighbors import KNeighborsClassifier

clfd = KNeighborsClassifier(n_neighbors = 15, weights = 'distance')
clfd.fit(train_x, train_y)

# 성능 평가
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score

pred = clfd.predict(test_x)

test_cm = confusion_matrix(test_y, pred)
test_acc = accuracy_score(test_y, pred)
test_prc = precision_score(test_y, pred)
test_rcll = recall_score(test_y, pred)
test_f1 = f1_score(test_y, pred)

print(test_cm)
print('\n')
print('정확도\t{}%'.format(round(test_acc * 100, 2)))
print('정밀도\t{}%'.format(round(test_prc * 100, 2)))
print('재현율\t{}%'.format(round(test_rcll * 100, 2)))
print('F1점수\t{}%'.format(round(test_f1 * 100, 2)))

[[103  21]
 [ 38  12]]


정확도	66.09%
정밀도	73.05%
재현율	83.06%
F1점수	77.74%


## KNN 회귀

In [18]:
# 샘플데이터 생성하기
import numpy as np
np.random.seed(0)
x = np.sort(5 * np.random.rand(400, 1), axis=0)
T = np.linspace(0, 5, 500)[:, np.newaxis]
y = np.sin(x).ravel()

# 타깃데이터에 노이즈 추가하기
y[::1] += 1 * (0.5 - np.random.rand(400))

# 데이터 분할
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(x, y,
                                                    train_size = 0.7,
                                                    random_state = 1)
print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

# 모델 학습
from sklearn.neighbors import KNeighborsRegressor
knn_uni = KNeighborsRegressor(n_neighbors = 20, weights = 'uniform')
knn_dis = KNeighborsRegressor(n_neighbors = 20, weights = 'distance')

knn_uni.fit(train_x, train_y)
knn_dis.fit(train_x, train_y)

(280, 1) (280,) (120, 1) (120,)


KNeighborsRegressor(n_neighbors=20, weights='distance')

In [20]:
# 성능 평가
uni_pred = knn_uni.predict(test_x)
dis_pred = knn_dis.predict(test_x)

from sklearn.metrics import mean_squared_error, mean_absolute_error

preds = [uni_pred, dis_pred]
weights = ['uniform', 'distance']
evals = ['mse', 'rmse', 'mae']

results = pd.DataFrame(index=weights, columns=evals)

for pred, nm in zip(preds, weights) :
  mse = mean_squared_error(test_y, pred)
  mae = mean_absolute_error(test_y, pred)
  rmse = np.sqrt(mse)

  results.loc[nm]['mse'] = round(mse, 2)
  results.loc[nm]['rmse'] = round(rmse, 2)
  results.loc[nm]['mae'] = round(mae, 2)

display(results)

,mse,rmse,mae
uniform,0.1,0.31,0.27
distance,0.11,0.34,0.28
